# Ozone: MERGED-UV

# Import packages¶

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-notebook")

# Define parameters

In [ ]:
region_slices = {
    "global": {"lat_slice": slice(-90, 90), "lon_slice": slice(0, 360)},
    "tropics": {"lat_slice": slice(-25, 25), "lon_slice": slice(0, 360)},
    "northern hemisphere mid-latitudes": {
        "lat_slice": slice(30, 60),
        "lon_slice": slice(0, 360),
    },
    "southern hemisphere mid-latitudes": {
        "lat_slice": slice(-30, -60),
        "lon_slice": slice(0, 360),
    },
    "northern hemisphere polar": {
        "lat_slice": slice(60, 90),
        "lon_slice": slice(0, 360),
    },
    "southern hemisphere polar": {
        "lat_slice": slice(-60, -90),
        "lon_slice": slice(0, 360),
    },
}

# Define request

In [ ]:
collection_id = "satellite-ozone-v1"

chunks = {"year": 1}

year_start = 1996
year_stop = 2022

varname = "total_ozone_column"

requests = {
    "format": "zip",
    "processing_level": "level_3",
    "variable": "atmosphere_mole_content_of_ozone",
    "sensor": "merged_uv",
    "vertical_aggregation": "total_column",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "version": "v2000",
}

# Define transform function:

In [ ]:
def spatial_weighted_mean(ds, lon_slice, lat_slice):
    ds = convert_source_to_time(ds)
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return diagnostics.spatial_weighted_mean(ds)


def time_weighted_mean(ds):
    ds = convert_source_to_time(ds)
    return diagnostics.time_weighted_mean(ds)


def convert_source_to_time(ds):
    # Naming convention: YYYYMM-*.nc
    ds["source"] = pd.to_datetime(ds["source"].str.slice(None, 6), format="%Y%m")
    return ds.rename(source="time")

# Download and transform data

In [ ]:
ds_map = download.download_and_transform(
    collection_id,
    requests,
    transform_func=convert_source_to_time,
    split_all=True,
)

dataarrays = []
for region, slices in region_slices.items():
    ds = download.download_and_transform(
        collection_id,
        requests,
        transform_func=spatial_weighted_mean,
        transform_func_kwargs=slices,
        chunks=chunks,
        drop_variables="time_bounds",
    )
    dataarrays.append(ds[varname].expand_dims(region=[region]))
da = xr.concat(dataarrays, "region")

DU_factor = ds_map["total_ozone_column"].attrs["multiplication_factor_to_convert_to_DU"]

# Results

## Data Coverage

In [ ]:
nobs_time = ds_map["total_ozone_column_number_of_observations"].sum(dim="time")
nobs_lat = ds_map["total_ozone_column_number_of_observations"].sum(dim="longitude")
nobs_lon = ds_map["total_ozone_column_number_of_observations"].sum(dim="latitude")

plot.projected_map(nobs_time, show_stats=False)

fig, axes = plt.subplots(2)
ax1 = axes[0].contourf(
    nobs_lat["time"], nobs_lat["latitude"], nobs_lat.transpose(), extend="max"
)
axes[0].set_ylabel("Latitude")
axes[0].set_xlabel("Year")

ax2 = axes[1].contourf(
    nobs_lon["time"], nobs_lon["longitude"], nobs_lon.transpose(), extend="max"
)
axes[1].set_ylabel("Longitude")
axes[1].set_xlabel("Year")

# plt.contourf(ds_obs_lat["time"],ds_obs_lat["latitude"],
#                         ds_obs_lat.transpose(),
#                         extend='both')

cbar = fig.colorbar(ax1, ax=axes.ravel().tolist(), shrink=0.95)
cbar.set_label("Mean number of observations %")

## Map of the Total Ozone column

In [ ]:
map = plot.projected_map(
    ds_map["total_ozone_column"].mean("time") * DU_factor,
    cmap="RdGy_r",
    levels=range(160, 460, 10),
    robust=True,
    extend="both",
    show_stats=False,
)

## Annual Cycle

In [ ]:
climatology_lon = ds_map.groupby("time.month").mean(["time", "longitude"])

fig, ax = plt.subplots()
im = ax.contourf(
    climatology_lon["total_ozone_column"]["month"],
    climatology_lon["total_ozone_column"]["latitude"],
    climatology_lon["total_ozone_column"].transpose("latitude", "month") * DU_factor,
    cmap="RdGy_r",
    extend="both",
    levels=range(160, 460, 10),
)
ax.set_title("TCO [DU]")
ax.set_ylabel("Latitude")
ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
ax.set_xticklabels(
    [
        "JAN",
        "FEB",
        "MAR",
        "APR",
        "MAY",
        "JUN",
        "JUL",
        "AUG",
        "SEP",
        "OCT",
        "NOV",
        "DEC",
    ],
    size=10,
)
fig.colorbar(im)

## Time series

In [ ]:
da_anomaly = DU_factor * (da.groupby("time.month") - da.groupby("time.month").mean())

fig = da_anomaly.plot(row="region", col_wrap=3)